In [137]:
%load_ext autoreload
import os
workDir = os.getcwd()
user = 'testID' #CERN user ID
institution = 'UCSB' #UCSB
print(workDir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/lbrennan/Desktop/Research/Coding/FiducialsQC-master


In [138]:
%run utils/GetExcel.ipynb

#The sensor file can contain the fiducials of the sensor, but can also have the 
#component heights for sensor in location P1 or P2.
#Methods can run either way! 

#pcbfile = workDir + '/data/some-PCB-file.xls'
pcbfile = workDir + '/data/LDV3D12_protomodule.xls'
fiducialfile = workDir + '/data/GantryTrayFiducialData.xls'
sensorfile = workDir + '/data/LDV3D12_protomodule.xls'
#sensorfile = workDir + '/data/sensorfiducialtrialrun.xls'
#surveyfile = workDir + '/data/alethea test.xls'
surveyfile = workDir + '/data/LDV3D12_protomodule.xls'


survey = GetExcel(surveyfile)
survey.read()
print(survey.df)

sensor = GetExcel(sensorfile)
sensor.read()
#print(sensor.df.tail())

fiducial = GetExcel(fiducialfile)
fiducial.read()
#print(fiducial.df.head())

pcb = GetExcel(pcbfile)
pcb.read()
#print(pcb.df.head())

     Unnamed: 0 Unnamed: 1              Unnamed: 2 Unnamed: 3 Unnamed: 4  \
3           NaN   Routine1                  mm  Ɵ°        NaN        NaN   
6           NaN        NaN                 Feature  Attribute    Nominal   
7           NaN        NaN           SensorCorner1          X    129.480   
8           NaN        NaN                     NaN          Y    352.284   
9           NaN        NaN                     NaN          Z      1.583   
..          ...        ...                     ...        ...        ...   
147         NaN        NaN  ProtomoduleThickness25          X    138.267   
148         NaN        NaN                     NaN          Y    265.346   
149         NaN        NaN                     NaN          Z      1.743   
152         NaN        NaN               Attribute     Actual        NaN   
153         NaN        NaN                   $PMID    LDV3D12        NaN   

    Unnamed: 5  Unnamed: 6  Unnamed: 7 Unnamed: 8  
3          NaN         NaN         

In [139]:
%run utils/ReadDataframe.ipynb
%autoreload 2

if sensor.exists:
    senDF = ReadDataframe(sensor.df, sensorfile)
    senDF.readComponentFiducials()
    senDF.readComponentHeights()
    print(senDF.readComponentHeights)

if fiducial.exists:
    fidDF = ReadDataframe(fiducial.df, fiducialfile)
    fidDF.readTrayFiducials()

if pcb.exists:
    pcbDF = ReadDataframe(pcb.df, pcbfile)
    pcbDF.readComponentFiducials()
    pcbDF.readComponentHeights()
    
if survey.exists:
    surveyDF = ReadDataframe(survey.df, surveyfile)
    surveyDF.readComponentHeights()

<bound method ReadDataframe.readComponentHeights of <__main__.ReadDataframe object at 0x7fb5fa43f5b0>>


In [140]:
%run utils/QualityControlLiam.ipynb
%autoreload 2

if sensor.exists:
    sensorP1 = QualityControlLiam(fidDF, senDF, 'P1', False)
    if len(senDF.pointsP1): sensorP1.runTransform()
    #if len(senDF.heightP1): sensorP1.runHeights()
    #LiamTest.RotationalOffset(self,distance)
    sensorP1.rating()
    print(sensorP1)

    sensorP2 = QualityControlLiam(fidDF, senDF, 'P2', False)
    if len(senDF.pointsP2): sensorP2.runTransform()
    #if len(senDF.heightP2): sensorP2.runHeights()
    sensorP2.rating()
    print(sensorP2)
    
if pcb.exists:
    pcbP1 = QualityControlLiam(fidDF, pcbDF, 'P1', True)
    if len(pcbDF.pointsP1): pcbP1.runTransform()
    #if len(pcbDF.heightP1): pcbP1.runHeights()
    pcbP1.rating()
    print(pcbP1)

    pcbP2 = QualityControlLiam(fidDF, pcbDF, 'P2', True)
    if len(pcbDF.pointsP2): pcbP2.runTransform()
    #if len(pcbDF.heightP2): pcbP2.runHeights()
    pcbP2.rating()
    print(pcbP2)
    
if survey.exists:
    surveyComp = QualityControl(fidDF, surveyDF, 'Survey', False, True)
    #if len(surveyDF.heightP1): surveyComp.runHeights()

The module/component in P1 has completed QC!

The FINAL rating of the module in position P1 is NO INFO.

The module/component in P2 has completed QC!

The FINAL rating of the module in position P2 is NO INFO.

The module/component in P1 has completed QC!

The FINAL rating of the module in position P1 is NO INFO.

The module/component in P2 has completed QC!

The FINAL rating of the module in position P2 is NO INFO.



In [141]:
%run utils/CreateXML.ipynb
%autoreload 2

if sensor.exists:
    if sensorP1.finalRating != 'NO INFO':
        sensorP1XML = CreateXML(sensorP1, user, institution)
        sensorP1XML.save(senDF.dir, senDF.name + '_P1.xml')

    if sensorP2.finalRating != 'NO INFO':
        sensorP2XML = CreateXML(sensorP2, user, institution)
        sensorP2XML.save(senDF.dir, senDF.name + '_P2.xml')
    
if pcb.exists:
    if pcbP1.finalRating != 'NO INFO':
        pcbP1XML = CreateXML(pcbP1, user, institution)
        pcbP1XML.save(pcbDF.dir, pcbDF.name + '_P1.xml')
    
    if pcbP2.finalRating != 'NO INFO':
        pcbP2XML = CreateXML(pcbP2, user, institution)
        pcbP2XML.save(pcbDF.dir, pcbDF.name + '_P2.xml')

In [142]:
%run utils/HeightPlotter.ipynb
%autoreload 2

if sensor.exists:
    if len(sensorP1.hgts):
        label = senDF.name + ' P1'
        senPlotP1 = HeightPlotter(sensorP1.hgts, sensorP1.hXY, senDF.name + '_P1', senDF.dir + '/', label)
        senPlotP1.padMapping()
    
    if len(sensorP2.hgts):
        label = senDF.name + ' P2'
        senPlotP2 = HeightPlotter(sensorP2.hgts, sensorP2.hXY, senDF.name + '_P2', senDF.dir + '/', label)
        senPlotP2.padMapping()

if pcb.exists:
    if len(pcbP1.hgts):
        label = pcbDF.name + ' P1'
        pcbPlotP1 = HeightPlotter(pcbP1.hgts, pcbP1.hXY, pcbDF.name + '_P1', pcbDF.dir + '/', label)
        pcbPlotP1.padMapping()
        
    if len(pcbP2.hgts):
        label = pcbDF.name + ' P2'
        pcbPlotP2 = HeightPlotter(pcbP2.hgts, pcbP2.hXY, pcbDF.name + '_P2', pcbDF.dir + '/', label)
        pcbPlotP2.padMapping()
        
if survey.exists:
    if len(surveyComp.hgts):
        label = surveyDF.name + ' Survey'
        surveyPlot = HeightPlotter(surveyComp.hgts, surveyComp.hXY, surveyDF.name + '_survey', surveyDF.dir + '/', label)
        surveyPlot.padMapping()

ModuleNotFoundError: No module named 'hexmap.plot_summary'

ModuleNotFoundError: No module named 'hexmap.plot_summary'